In [1]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as f 
from torch.utils.data import DataLoader 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 

# Set device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set Hyperparameters 
input_size = 28 
sequenc_length = 28 
num_layers = 2 
hidden_size = 256 
num_classes = 10 
learning_rate = 0.001 
batch_size = 64 
num_epochs = 4
load_model = False

In [20]:
# Fine tuning and improve the model (accuracy), Modify
import sys 
import torchvision 

class Identity(nn.Module):
    def __init__(self): 
        super(Identity, self).__init__() 
        
    def forward(self, x): 
        return x 

model = torchvision.models.vgg16(pretrained=True)

for param in model.parameters(): 
    param.requires_grad = False 
    
    
    
model.avgpool = Identity() 
model.classifier = nn.Sequential(nn.Linear(512, 10),
                                 nn.ReLU(),
                                 nn.Linear(100, 10))
# model.classifier = nn.Linear(512, 10)
model.to(device)
# print(model)

# sys.exit()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [13]:
class BRNN(nn.Module): 
    def __init__(self, input_zie, hidden_size, num_layers, num_classes): 
        super(BRNN, self).__init__()
        self.hidden_size = hidden_size 
        self.num_layers = num_layers 
        self.lstm = nn.LSTM(input_zie, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        
    def forward(self, x): 
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        
        return out

In [66]:
def save_checkpoint(state, filename="checkpoint.pth.tar"):
    print("saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint): 
    print(" loaded checkpoint")
    model.load_state_dict(chechpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    

In [62]:
# Initialize Network
model = BRNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Lass and Optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

if load_model: 
    load_model(torch.load("my_checkpoint.pth.tar"))

In [63]:
# Load data 
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


In [67]:

# Train the Network 
for epoch in range(num_epochs): 
    losses = [] 
    
    if epoch == 2: 
        print(f"Save at epoch {epoch}")
        checkpoint = {'state_dict' : model.state_dict(), 'optimizer' : optimizer.state_dict()}
        save_checkpoint(checkpoint)
        
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)
        
        # forward 
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient desent or adam step
        optimizer.step()
    
    mean_loss = sum(losses)/len(losses)
    print(f' Loss at epoch {epoch} was {mean_loss:.5f}')
        
        
# Check accuracy 
def check_accuracy(loader, model): 
    if loader.dataset.train: 
        print("Checking accuracy on training data...")
    else:
        print("Checking accuracy on test data...")
        
    num_correct = 0 
    num_samples = 0 
    model.eval() 
    
    with torch.no_grad(): 
        for x, y in loader: 
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)
            
            
            scores = model(x)  # RNN()
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum() 
            num_samples += predictions.size(0)
            
        print(f'got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100}')
        
    model.train()


 Loss at epoch 0 was 0.06819
 Loss at epoch 1 was 0.05119
Save at epoch 2
saving checkpoint
 Loss at epoch 2 was 0.04382
 Loss at epoch 3 was 0.03489


In [ ]:
nvidia-smi